In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from src.config import OPENAIVARS
import json
from functools import partial
from src.genai_assistance.format_document import llamaindex_format_doc
from src.gradio.business_chatbot import business_chat
from contracts.business_listings.prompt_contract import ChatPrompts
from llama_index.core import VectorStoreIndex
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.openai import OpenAI
from llama_index.core.chat_engine import CondensePlusContextChatEngine
import gradio as gr

In [7]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [8]:
openai_model = OPENAIVARS.open_ai_model

### Define fixed variables

In [9]:
ROOT_DIR = os.path.dirname(os.path.abspath("__file__")) 

### Read Data

In [10]:
with open( os.path.join(ROOT_DIR, "data", "genai_extracted_info.json"), "r") as f:
    genai_augmented_listings_ = json.load(f)  

In [11]:
vector_store_listings = []

for listing in genai_augmented_listings_:
    try:
        filtered_listing = {k: v for k, v in listing.items() if k != "raw_content"}
        extracted_info = filtered_listing.get("extracted_info", {})
        extracted_info.pop("Recommendation", None)
        filtered_listing["extracted_info"] = extracted_info

        vector_store_listings.append(filtered_listing)

    except KeyError:
        print(listing)
        pass


## Build List of Documents

In [12]:
docs = [llamaindex_format_doc(listing) for listing in vector_store_listings]

## Create Vector Store

In [13]:
index = VectorStoreIndex.from_documents(docs) # no chunking. Each listing has its own embedding [cleaned content]

## Set up chatbot components

In [ ]:
llm = OpenAI(model=openai_model, temperature=0,streaming = True)

retriever = index.as_retriever(similarity_top_k=100)

memory = ChatMemoryBuffer.from_defaults(token_limit=1000)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    llm=llm,
    memory=memory,
    system_prompt=ChatPrompts.CHATBOT_SYSTEM_PROMPT)

## Chatbot

In [40]:
iface = gr.ChatInterface(
    fn=partial(business_chat, llm_chat_engine=chat_engine),
    title="Business Advisor Chat"
)

iface.launch()


c:\Users\marce\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
